In [2]:
import pickle
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from typing import List

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

    
from src import utils


In [3]:
# Cargando el modelo:
model_sales = pickle.load(open('pickles/model.pkl', 'rb'))

#cargando scaler
scaler = pickle.load(open('pickles/scaler_model.pkl', 'rb'))

#cargando encoder
encoder = pickle.load(open('pickles/encoder.pkl', 'rb'))

d:\Programas\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Programas\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.2.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
ross_df = pd.read_csv('data/train.csv',low_memory=False)
store_df = pd.read_csv('data/store.csv')
merged_df = ross_df.merge(store_df, how='left', on='Store')

df = merged_df[merged_df["Open"] == 1].copy()

In [5]:
print(ross_df.shape)
ross_df.head(3)

(1017209, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1


In [27]:
ross_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [6]:
df.isna().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2186
CompetitionOpenSinceMonth    268619
CompetitionOpenSinceYear     268619
Promo2                            0
Promo2SinceWeek              423307
Promo2SinceYear              423307
PromoInterval                423307
dtype: int64

In [7]:
ross_df.DayOfWeek.value_counts()

5    145845
4    145845
3    145665
2    145664
1    144730
7    144730
6    144730
Name: DayOfWeek, dtype: int64

In [8]:
store_df.head(3)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [26]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [9]:
store_df.shape

(1115, 10)

In [10]:
#Variables del modelo:
var_final = ['Store', 'Promo', 'SchoolHoliday', 'CompetitionDistance',
       'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month', 'Day',
       'Month', 'Year', 'WeekOfYear', 'DayOfWeek_1', 'DayOfWeek_2',
       'DayOfWeek_3', 'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6',
       'DayOfWeek_7', 'StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b',
       'StateHoliday_c', 'StoreType_a', 'StoreType_b', 'StoreType_c',
       'StoreType_d', 'Assortment_a', 'Assortment_b', 'Assortment_c']

In [11]:
#Store: Lo da el usuario
#Promo: Lo da el usuario
#SchoolHoliday: Lo da el usuario
#StateHoliday: Lo da el usuario
#Date: lo da el usuario y sale otras variables

#depende de date:
#Day : calculada de date
#Month: calculada de date
#Year : Calculada de date
#WeekOfYear: Calculada de date
#DayOfWeek: Calculada de date (new logic)


## Todo lo relacionado a la tienda, podemos extraer información directamente
#CompetitionDistance: direto de la base de tienda

#Promo2: direto de la base de tienda
#StoreType: direto de la base de tienda
#Assortment: direto de la base de tienda




#calculada
#CompetitionOpen: calculada: (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)

#Promo2Open: es de un feature engineering: Year y WeekOfYear (ambos calculadas de Date),Promo2SinceYear - Promo2SinceWeek-  Promo2 (base store)
#IsPromo2Month : Sale de Promo2Open y  Promo2 (ambos bd store)

#van a ayudar para calcular las otras variables
#CompetitionOpenSinceYear
#CompetitionOpenSinceMonth
#Promo2SinceYear
#Promo2SinceWeek







In [12]:
var_final = ['Store', 'Date','Promo', 'SchoolHoliday', 'StateHoliday',
             'CompetitionDistance','Promo2', 'StoreType','Assortment',
             'CompetitionOpenSinceYear','CompetitionOpenSinceMonth','Promo2SinceYear',
             'Promo2SinceWeek','PromoInterval',
             'Sales']

In [13]:
merged_df = ross_df.merge(store_df, how='left', on='Store')
print(merged_df.shape)
df = merged_df[merged_df["Open"] == 1].copy()
df = df[var_final]
print(df.shape)
df.head()

(1017209, 18)
(844392, 15)


,Store,Date,Promo,SchoolHoliday,StateHoliday,CompetitionDistance,Promo2,StoreType,Assortment,CompetitionOpenSinceYear,CompetitionOpenSinceMonth,Promo2SinceYear,Promo2SinceWeek,PromoInterval,Sales
0,1,2015-07-31,1,1,0,1270.0,0,c,a,2008.0,9.0,NaN,NaN,NaN,5263
1,2,2015-07-31,1,1,0,570.0,1,a,a,2007.0,11.0,2010.0,13.0,"Jan,Apr,Jul,Oct",6064
2,3,2015-07-31,1,1,0,14130.0,1,a,a,2006.0,12.0,2011.0,14.0,"Jan,Apr,Jul,Oct",8314
3,4,2015-07-31,1,1,0,620.0,0,c,c,2009.0,9.0,NaN,NaN,NaN,13995
4,5,2015-07-31,1,1,0,29910.0,0,a,a,2015.0,4.0,NaN,NaN,NaN,4822


In [14]:
#utils.split_date(df)

## Funciones

In [15]:
def split_date(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['WeekOfYear'] = df.Date.dt.isocalendar().week
    df['DayOfWeek'] = df.Date.dt.dayofweek + 1
    df['DayOfWeek'] = df['DayOfWeek'].astype(int)
    
def comp_months(df):
    df['CompetitionOpen'] = 12 * (df.Year - df.CompetitionOpenSinceYear) + (df.Month - df.CompetitionOpenSinceMonth)
    df['CompetitionOpen'] = df['CompetitionOpen'].map(lambda x: 0 if x < 0 else x).fillna(0)
    
def check_promo_month(row):
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',              
                 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    try:
        months = (row['PromoInterval'] or '').split(',')
        if row['Promo2Open'] and month2str[row['Month']] in months:
            return 1
        else:
            return 0
    except Exception:
        return 0

def promo_cols(df):
    # Months since Promo2 was open
    df['Promo2Open'] = 12 * (df.Year - df.Promo2SinceYear) +  (df.WeekOfYear - df.Promo2SinceWeek)*7/30.5
    df['Promo2Open'] = df['Promo2Open'].map(lambda x: 0 if x < 0 else x).fillna(0) * df['Promo2']
    # Whether a new round of promotions was started in the current month
    df['IsPromo2Month'] = df.apply(check_promo_month, axis=1) * df['Promo2']    
    
def input_var(df):
    max_distance = 75860

    df['CompetitionDistance'].fillna(max_distance, inplace=True)

    

In [16]:
split_date(df)
comp_months(df)
promo_cols(df)


In [17]:
df.head(3)

,Store,Date,Promo,SchoolHoliday,StateHoliday,CompetitionDistance,Promo2,StoreType,Assortment,CompetitionOpenSinceYear,...,PromoInterval,Sales,Year,Month,Day,WeekOfYear,DayOfWeek,CompetitionOpen,Promo2Open,IsPromo2Month
0,1,2015-07-31,1,1,0,1270.0,0,c,a,2008.0,...,NaN,5263,2015,7,31,31,5,82.0,0.000000,0
1,2,2015-07-31,1,1,0,570.0,1,a,a,2007.0,...,"Jan,Apr,Jul,Oct",6064,2015,7,31,31,5,92.0,64.131148,1
2,3,2015-07-31,1,1,0,14130.0,1,a,a,2006.0,...,"Jan,Apr,Jul,Oct",8314,2015,7,31,31,5,103.0,51.901639,1


In [168]:
# Aplicamos el modelo

In [18]:
#variables:
input_cols = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 
              'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpen', 
              'Day', 'Month', 'Year', 'WeekOfYear',  'Promo2', 
              'Promo2Open', 'IsPromo2Month','Sales']


numeric_cols = ['Store', 'Promo', 'SchoolHoliday', 
              'CompetitionDistance', 'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month',
              'Day', 'Month', 'Year', 'WeekOfYear']
categorical_cols = ['DayOfWeek', 'StateHoliday', 'StoreType', 'Assortment']


In [19]:
df = df[input_cols]#.reset_index(drop=True)       

df[numeric_cols] = scaler.transform(df[numeric_cols])

encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
df[encoded_cols] = encoder.transform(df[categorical_cols])

var_final = ['Store', 'Promo', 'SchoolHoliday', 'CompetitionDistance',
       'CompetitionOpen', 'Promo2', 'Promo2Open', 'IsPromo2Month', 'Day',
       'Month', 'Year', 'WeekOfYear', 'DayOfWeek_1', 'DayOfWeek_2',
       'DayOfWeek_3', 'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6',
       'DayOfWeek_7', 'StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b',
       'StateHoliday_c', 'StoreType_a', 'StoreType_b', 'StoreType_c',
       'StoreType_d', 'Assortment_a', 'Assortment_b', 'Assortment_c']
    


In [20]:
df['preds'] = model_sales.predict(df[var_final])
df.head(3)

print(df.columns)
df.head(3)

Index(['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpen',
       'Day', 'Month', 'Year', 'WeekOfYear', 'Promo2', 'Promo2Open',
       'IsPromo2Month', 'Sales', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c',
       'StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d',
       'Assortment_a', 'Assortment_b', 'Assortment_c', 'preds'],
      dtype='object')


,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpen,Day,...,StateHoliday_b,StateHoliday_c,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,preds
0,0.000000,5,1.0,0,1.0,c,a,0.016482,0.059163,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,5388.752930
1,0.000898,5,1.0,0,1.0,a,a,0.007252,0.066378,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,6054.424316
2,0.001795,5,1.0,0,1.0,a,a,0.186050,0.074315,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8427.394531


In [21]:
df[['Store', 'DayOfWeek','DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7']].tail(3)

,Store,DayOfWeek,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7
1016863,0.689408,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1017042,0.850090,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1017190,0.983842,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df[['Sales','preds']].head(3)

,Sales,preds
0,5263,5388.752930
1,6064,6054.424316
2,8314,8427.394531


In [23]:
from sklearn.metrics import mean_squared_error

def rmse(a, b):
    return mean_squared_error(a, b, squared=False)

In [24]:
rmse(df['preds'], df['Sales'])

443.22155943861503